Replicate [Dynamic Return Dependencies Across Industries: A Machine Learning Approach](https://papers.ssrn.com/sol3/papers.cfm?abstract_id=3120110&download=yes) by David Rapach, Jack Strauss, Jun Tu and Guofu Zhou.

1) Use industry returns from [Ken French](http://mba.tuck.dartmouth.edu/pages/faculty/ken.french/data_library.html)

2) Forecast (for example) this month's Chemical industry return using last month's returns from all 30 industries 

3) Use LASSO for predictor subset selection over the entire 1960-2016 period to determine that e.g. Beer is predicted by Food, Clothing, Coal

4) Use those predictors and simple linear regression to predict returns

5) Generate portfolios and run backtests.

- Predictor selection - finds same predictors except 2 industries. Possibly use of AICc instead of AIC (don't see an sklearn implementation that uses AICc)

- Prediction by industry - R-squareds line up pretty closely

- Portfolio performance, similar ballpark results. Since prediction is similar but return profile is different, must be some difference in portfolio construction. (am taking equal weight top 6 predicted as long and bottom 6 as short, every month)

- For some reason their mean returns don't line up to geometric mean annualized, they seem to be calculating something different.

- But it does replicate closely and perform well

 

In [1]:
import os
import sys
import warnings
import numpy as np
import pandas as pd
import time 
import copy
import random
from itertools import product

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' #Hide messy TensorFlow warnings
warnings.filterwarnings("ignore") #Hide messy numpy warnings

from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error, explained_variance_score, r2_score
from sklearn.linear_model import LinearRegression, Lasso, lasso_path, lars_path, LassoLarsIC
from sklearn.ensemble.forest import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import MinMaxScaler, StandardScaler

import ffn
%matplotlib inline

import plotly as py
# print (py.__version__) # requires version >= 1.9.0
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.graph_objs import *
import plotly.figure_factory as ff

init_notebook_mode(connected=True)

random.seed(1764)
np.random.seed(1764)


In [2]:
print("Loading data...")
data = pd.read_csv("30_Industry_Portfolios.csv")
data = data.set_index('yyyymm')
industries = list(data.columns)
# map industry names to col nums
ind_reverse_dict = dict([(industries[i], i) for i in range(len(industries))])

rfdata = pd.read_csv("F-F_Research_Data_Factors.csv")
rfdata = rfdata.set_index('yyyymm')
data['rf'] = rfdata['RF']

# subtract risk-free rate
# create a response variable led by 1 period to predict
for ind in industries:
    data[ind] = data[ind] - data['rf']

#for ind in industries:
#    data[ind+".3m"] = pd.rolling_mean(data[ind],3)
    
#for ind in industries:
#    data[ind+".6m"] = pd.rolling_mean(data[ind],6)

#for ind in industries:
#    data[ind+".12m"] = pd.rolling_mean(data[ind],12)
    
for ind in industries:
    data[ind+".lead"] = data[ind].shift(-1)

data = data.loc[data.index[data.index > 195911]]
data = data.drop(columns=['rf'])    
data = data.dropna(axis=0, how='any')

nresponses = len(industries)
npredictors = data.shape[1]-nresponses

predictors = list(data.columns[:npredictors])
predictor_reverse_dict = dict([(predictors[i], i) for i in range(len(predictors))])

responses = list(data.columns[-nresponses:])
response_reverse_dict = dict([(responses[i], i) for i in range(len(responses))])

print(data.shape)

data[['Food', 'Food.lead']]


Loading data...
(697, 60)


,Food,Food.lead
yyyymm,,
195912,2.01,-4.49
196001,-4.49,3.35
196002,3.35,-1.67
196003,-1.67,1.17
196004,1.17,8.20
196005,8.20,5.39
196006,5.39,-2.11
196007,-2.11,4.57
196008,4.57,-3.88


In [3]:
# exclude 2017 and later to tie to paper
data = data.loc[data.index[data.index < 201701]]
data = data.loc[data.index[data.index > 195911]]
data


,Food,Beer,Smoke,Games,Books,Hshld,Clths,Hlth,Chems,Txtls,...,Telcm.lead,Servs.lead,BusEq.lead,Paper.lead,Trans.lead,Whlsl.lead,Rtail.lead,Meals.lead,Fin.lead,Other.lead
yyyymm,,,,,,,,,,,,,,,,,,,,,
195912,2.01,0.35,-3.02,1.64,7.29,0.67,1.87,-1.97,3.08,0.74,...,0.62,-6.18,-7.93,-9.41,-4.31,-5.33,-6.09,-10.08,-4.68,-3.98
196001,-4.49,-5.71,-2.05,1.21,-5.47,-7.84,-8.53,-6.68,-10.03,-4.77,...,8.07,9.13,5.09,3.00,-0.94,1.42,4.00,1.81,-0.98,6.32
196002,3.35,-2.14,2.27,4.23,2.39,9.31,1.44,-0.02,-0.74,0.32,...,-0.21,-0.31,3.34,-2.43,-4.99,-1.37,-0.13,-3.88,0.05,-2.43
196003,-1.67,-2.94,-0.18,-0.65,2.18,-0.56,-2.59,1.26,-2.75,-6.79,...,-1.24,7.14,1.77,0.41,-2.13,0.45,-0.53,8.86,-0.64,0.55
196004,1.17,-2.16,1.35,6.46,-1.17,-1.27,0.21,1.49,-5.53,-1.10,...,3.05,-1.75,11.90,2.85,0.90,1.65,3.11,0.80,-0.45,1.02
196005,8.20,-0.52,2.44,7.28,11.67,7.74,1.74,13.50,3.40,2.10,...,-0.58,-8.07,2.39,3.50,2.17,5.96,3.41,1.03,3.72,6.41
196006,5.39,0.47,4.73,2.24,0.02,6.38,-1.59,-0.40,0.45,4.04,...,-0.03,2.84,-2.02,-4.10,-3.11,-6.16,-2.99,-1.25,0.09,-5.95
196007,-2.11,-0.79,4.60,-4.72,0.23,-0.60,-1.10,-3.99,-6.80,-3.14,...,6.94,5.69,2.71,1.18,1.98,4.51,2.85,2.05,3.47,3.48
196008,4.57,3.24,5.20,7.16,3.63,5.09,3.34,2.29,1.17,-0.84,...,-6.07,-3.53,-7.61,-7.37,-7.07,-8.44,-8.57,-1.90,-5.78,-4.21


In [4]:
data.to_csv("data.csv")
desc = data.describe()
desc
# min, max line up with Table 1

,Food,Beer,Smoke,Games,Books,Hshld,Clths,Hlth,Chems,Txtls,...,Telcm.lead,Servs.lead,BusEq.lead,Paper.lead,Trans.lead,Whlsl.lead,Rtail.lead,Meals.lead,Fin.lead,Other.lead
count,685.000000,685.000000,685.000000,685.000000,685.000000,685.000000,685.000000,685.000000,685.000000,685.000000,...,685.000000,685.000000,685.000000,685.000000,685.000000,685.000000,685.000000,685.000000,685.000000,685.000000
mean,0.690715,0.710613,0.982321,0.701708,0.528277,0.554190,0.669460,0.650905,0.519781,0.667416,...,0.520847,0.694234,0.584175,0.511241,0.582088,0.625562,0.662219,0.702730,0.609810,0.385620
std,4.339811,5.090215,6.061582,7.180918,5.809314,4.759874,6.386027,4.928072,5.518477,7.022552,...,4.628520,6.527984,6.738979,5.055314,5.739306,5.605317,5.349341,6.104515,5.411766,5.815446
min,-18.150000,-20.190000,-25.320000,-33.400000,-26.560000,-22.240000,-31.500000,-21.060000,-28.600000,-33.110000,...,-16.440000,-28.670000,-32.070000,-27.740000,-28.500000,-29.250000,-29.740000,-31.890000,-22.530000,-28.090000
25%,-1.640000,-2.100000,-2.780000,-3.490000,-2.690000,-2.110000,-2.810000,-2.240000,-2.800000,-3.200000,...,-2.110000,-3.090000,-3.290000,-2.430000,-2.780000,-2.570000,-2.430000,-2.940000,-2.420000,-2.990000
50%,0.740000,0.710000,1.280000,0.890000,0.510000,0.750000,0.690000,0.750000,0.670000,0.630000,...,0.610000,0.970000,0.560000,0.690000,0.860000,0.940000,0.470000,1.030000,0.820000,0.470000
75%,3.120000,3.660000,4.640000,5.310000,3.720000,3.550000,4.310000,3.560000,3.760000,4.490000,...,3.360000,4.290000,4.590000,3.460000,4.060000,3.880000,4.000000,4.330000,4.000000,4.200000
max,19.890000,25.510000,32.380000,34.520000,33.130000,18.220000,31.790000,29.010000,21.680000,59.030000,...,21.220000,23.380000,24.660000,21.000000,18.500000,17.530000,26.490000,27.380000,20.590000,19.960000


In [5]:
# annualized returns don't match Table 1, oddly
# geometric mean, annualized
pd.DataFrame((np.prod(data/100 + 1)**(12.0/len(data))-1)[:30], columns=['Mean Ann. Return'])

,Mean Ann. Return
Food,0.074020
Beer,0.072005
Smoke,0.100147
Games,0.054031
Books,0.043953
Hshld,0.054098
Clths,0.057170
Hlth,0.065463
Chems,0.044917
Txtls,0.051888


In [6]:
# try this way, arithmetic mean then annualize (not very correct)
#print(pd.DataFrame(((desc.loc['mean']/100+1)**12-1)[:30]))
#nope

# same
pd.DataFrame(((1 + np.mean(data, axis=0)/100)**12 -1)[:30], columns=['Mean Ann. Return'])

,Mean Ann. Return
Food,0.086108
Beer,0.088687
Smoke,0.124460
Games,0.087532
Books,0.065268
Hshld,0.068568
Clths,0.083360
Hlth,0.080966
Chems,0.064188
Txtls,0.083096


In [7]:
#annualized volatility 
pd.DataFrame((desc.loc['std']*np.sqrt(12))[:30].round(2))
# lines up with table 1

,std
Food,15.03
Beer,17.63
Smoke,21.00
Games,24.88
Books,20.12
Hshld,16.49
Clths,22.12
Hlth,17.07
Chems,19.12
Txtls,24.33


In [8]:
# Run LASSO, then OLS on selected variables

# skip last row to better match published r-squared
# looks like they forecast actuals 1960-2016 using 1959m12 to 2016m11
# not exact matches to Table 2 R-squared but almost within rounding error 
X = data.values[:-1,:npredictors]
Y = data.values[:-1,-nresponses:]
nrows = X.shape[0]
X.shape

(684, 30)

In [9]:
def subset_selection(X, Y, model_aic, verbose=False, responses=responses, predictors=predictors):
    
    nrows, npreds = X.shape
    nows, nresps = Y.shape
    coef_dict = []
    
    for response_index in range(nresps):
        y = Y[:,response_index]
        model_aic.fit(X, y)
        predcols = [i for i in range(npreds) if model_aic.coef_[i] !=0]

        #y_response = model_aic.predict(X)
        # print ("In-sample LASSO R-squared: %.6f" % r2_score(y, y_response))
        if verbose and responses:
            print("LASSO variables selected for %s: " % responses[response_index])
            print([predictors[i] for i in predcols])
        
        if not predcols:
            if verbose and responses:
                print("No coefs selected for " + responses[response_index] + ", using all")
                print("---")
            predcols = list(range(npreds))    
            
        # fit OLS vs. selected vars, better fit w/o LASSO penalties
        # in-sample R-squared using LASSO coeffs
        coef_dict.append(predcols)
        if verbose and responses and predictors:
            print("Running OLS for " + responses[response_index] + " against " + str([predictors[i] for i in predcols]))
            # col nums of selected responses
            model_ols = LinearRegression()
            model_ols.fit(X[:, predcols], y)
            y_pred = model_ols.predict(X[:, predcols])
            print ("In-sample OLS R-squared: %.2f%%" % (100 * r2_score(y, y_pred)))
            print("---")
            
    return coef_dict

#coef_dict = subset_selection(X, Y, LassoLarsIC(criterion='aic'))
coef_dict = subset_selection(X, Y, LassoLarsIC(criterion='aic'), verbose=True, responses=responses, predictors=predictors)
print(coef_dict)
# These subsets line up closely with Table 2
# except Clths, Whlsl, we get different responses

LASSO variables selected for Food.lead: 
['Clths', 'Coal', 'Util', 'Rtail']
Running OLS for Food.lead against ['Clths', 'Coal', 'Util', 'Rtail']
In-sample OLS R-squared: 2.24%
---
LASSO variables selected for Beer.lead: 
['Food', 'Clths', 'Coal']
Running OLS for Beer.lead against ['Food', 'Clths', 'Coal']
In-sample OLS R-squared: 2.52%
---
LASSO variables selected for Smoke.lead: 
['Txtls', 'Carry', 'Mines', 'Coal', 'Oil', 'Util', 'Telcm', 'Servs', 'Paper', 'Trans', 'Fin']
Running OLS for Smoke.lead against ['Txtls', 'Carry', 'Mines', 'Coal', 'Oil', 'Util', 'Telcm', 'Servs', 'Paper', 'Trans', 'Fin']
In-sample OLS R-squared: 6.55%
---
LASSO variables selected for Games.lead: 
['Books', 'Clths', 'Coal', 'Fin']
Running OLS for Games.lead against ['Books', 'Clths', 'Coal', 'Fin']
In-sample OLS R-squared: 5.05%
---
LASSO variables selected for Books.lead: 
['Games', 'Books', 'Coal', 'Oil', 'Util', 'Servs', 'BusEq', 'Rtail', 'Fin']
Running OLS for Books.lead against ['Games', 'Books', 'Coal'

In [10]:
# same predictors selected for all but 2 response vars
# use predictors from paper to match results
if True: # turn off/on
    coef_dict_temp = {}
    coef_dict_temp['Food.lead'] = ['Clths', 'Coal', 'Util', 'Rtail']
    coef_dict_temp['Beer.lead'] = ['Food', 'Clths', 'Coal']
    coef_dict_temp['Smoke.lead'] = ['Txtls', 'Carry', 'Mines', 'Coal', 'Oil', 'Util', 'Telcm', 'Servs', 'Paper', 'Trans', 'Fin']
    coef_dict_temp['Games.lead'] = ['Books', 'Clths', 'Coal', 'Fin']
    coef_dict_temp['Books.lead'] = ['Games', 'Books', 'Coal', 'Oil', 'Util', 'Servs', 'BusEq', 'Rtail', 'Fin']
    coef_dict_temp['Hshld.lead'] = ['Clths', 'Coal', 'Rtail']
    coef_dict_temp['Clths.lead'] = ['Books', 'Clths', 'Chems', 'Steel', 'ElcEq', 'Carry',  'Coal', 'Oil', 'Util','Telcm', 'Servs', 'BusEq', 'Rtail']
    # Running OLS for Clths against ['Clths', 'Coal', 'Oil', 'Servs', 'Rtail']
    coef_dict_temp['Hlth.lead'] = ['Books', 'Mines', 'Coal', 'Util']
    coef_dict_temp['Chems.lead'] = ['Clths']
    coef_dict_temp['Txtls.lead'] = ['Clths', 'Autos', 'Coal', 'Oil', 'Rtail', 'Fin']
    coef_dict_temp['Cnstr.lead'] = ['Clths', 'Coal', 'Oil', 'Util', 'Trans', 'Rtail', 'Fin']
    coef_dict_temp['Steel.lead'] = ['Fin']
    coef_dict_temp['FabPr.lead'] = ['Trans', 'Fin']
    coef_dict_temp['ElcEq.lead'] = ['Fin']
    coef_dict_temp['Autos.lead'] = ['Hshld', 'Clths', 'Coal', 'Oil', 'Util', 'BusEq', 'Rtail', 'Fin']
    coef_dict_temp['Carry.lead'] = ['Trans']
    coef_dict_temp['Mines.lead'] = []
    coef_dict_temp['Coal.lead'] = ['Beer', 'Smoke', 'Books', 'Autos', 'Coal', 'Oil', 'Paper', 'Rtail']
    coef_dict_temp['Oil.lead'] = ['Beer', 'Hlth', 'Carry']
    coef_dict_temp['Util.lead'] = ['Food', 'Beer', 'Smoke', 'Hshld', 'Hlth', 'Cnstr', 'FabPr', 'Carry', 'Mines', 'Oil', 'Util', 'Telcm', 'BusEq', 'Whlsl', 'Fin', 'Other']
    coef_dict_temp['Telcm.lead'] = ['Beer', 'Smoke', 'Books', 'Hshld', 'Cnstr', 'Autos', 'Carry', 'Mines', 'Coal', 'Oil', 'Util', 'Servs', 'BusEq', 'Rtail', 'Meals', 'Fin']
    coef_dict_temp['Servs.lead'] = ['Smoke', 'Books', 'Steel', 'Oil', 'Util', 'Fin']
    coef_dict_temp['BusEq.lead'] = ['Smoke', 'Books', 'Util']
    coef_dict_temp['Paper.lead'] = ['Clths', 'Coal', 'Oil', 'Rtail', 'Fin']
    coef_dict_temp['Trans.lead'] = ['Fin']
    coef_dict_temp['Whlsl.lead'] = ['Food', 'Beer', 'Smoke', 'Books', 'Hlth', 'Carry', 'Coal', 'Oil', 'Util', 'Telcm', 'Servs', 'BusEq', 'Fin', 'Other']
    # Running OLS for Whlsl against ['Food', 'Smoke', 'Books', 'Carry', 'Coal', 'Oil', 'Util', 'Servs', 'Fin', 'Other']
    coef_dict_temp['Rtail.lead'] = ['Rtail']
    coef_dict_temp['Meals.lead'] = ['Smoke', 'Books', 'Clths', 'Steel', 'Carry', 'Coal', 'Oil', 'Util', 'Servs', 'BusEq', 'Meals', 'Fin']
    coef_dict_temp['Fin.lead'] = ['Fin']
    coef_dict_temp['Other.lead'] = ['Clths', 'Fin']
    
coef_dict_paper = []
for response in coef_dict_temp.keys():
    print(response, " -> ", coef_dict_temp[response])
    coef_dict_paper.append([predictor_reverse_dict[jstr] for jstr in coef_dict_temp[response]])
print(coef_dict_paper)

Food.lead  ->  ['Clths', 'Coal', 'Util', 'Rtail']
Beer.lead  ->  ['Food', 'Clths', 'Coal']
Smoke.lead  ->  ['Txtls', 'Carry', 'Mines', 'Coal', 'Oil', 'Util', 'Telcm', 'Servs', 'Paper', 'Trans', 'Fin']
Games.lead  ->  ['Books', 'Clths', 'Coal', 'Fin']
Books.lead  ->  ['Games', 'Books', 'Coal', 'Oil', 'Util', 'Servs', 'BusEq', 'Rtail', 'Fin']
Hshld.lead  ->  ['Clths', 'Coal', 'Rtail']
Clths.lead  ->  ['Books', 'Clths', 'Chems', 'Steel', 'ElcEq', 'Carry', 'Coal', 'Oil', 'Util', 'Telcm', 'Servs', 'BusEq', 'Rtail']
Hlth.lead  ->  ['Books', 'Mines', 'Coal', 'Util']
Chems.lead  ->  ['Clths']
Txtls.lead  ->  ['Clths', 'Autos', 'Coal', 'Oil', 'Rtail', 'Fin']
Cnstr.lead  ->  ['Clths', 'Coal', 'Oil', 'Util', 'Trans', 'Rtail', 'Fin']
Steel.lead  ->  ['Fin']
FabPr.lead  ->  ['Trans', 'Fin']
ElcEq.lead  ->  ['Fin']
Autos.lead  ->  ['Hshld', 'Clths', 'Coal', 'Oil', 'Util', 'BusEq', 'Rtail', 'Fin']
Carry.lead  ->  ['Trans']
Mines.lead  ->  []
Coal.lead  ->  ['Beer', 'Smoke', 'Books', 'Autos', 'Coal', 

In [11]:
def predict_with_subsets(X, Y, create_model, coef_dict, verbose=False):
    """evaluate subset selection, pass a model function and subsets, compute avg R-squared"""
    global responses

    nrows, ncols = Y.shape
    model = create_model()
    
    scores = []
    for response_col in range(ncols):
        y = Y[:,response_col]

#        print("LASSO variables selected for %s: " % pred)
#        print(coef_dict[pred])
        
        if not coef_dict[response_col]:
            if verbose:
                print("No coefs selected for " + responses[response_col])
 #           print("---")
            continue
        # fit model vs. selected vars, better fit w/o LASSO penalties
        # in-sample R-squared using LASSO coeffs
        #print("Running model for " + pred + " against " + str(coef_dict[pred]))
        # col nums of selected predictors
        predcols = coef_dict[response_col]
        model.fit(X[:, predcols], y)
        y_pred = model.predict(X[:, predcols])
        score = r2_score(y, y_pred)
        scores.append(score)
        if verbose:
            print ("In-sample R-squared: %.2f%% for %s against %s" % (score*100, responses[response_col], 
                                                                      str([predictors[i] for i in coef_dict[response_col]])))
#        print("---")
    
    if verbose:
        print("Mean R-squared: %.2f%%" % (100 * np.mean(np.array(scores))))
    return np.mean(np.array(scores))
    
predict_with_subsets(X, Y, LinearRegression, coef_dict_paper, verbose=True)


In-sample R-squared: 2.24% for Food.lead against ['Clths', 'Coal', 'Util', 'Rtail']
In-sample R-squared: 2.52% for Beer.lead against ['Food', 'Clths', 'Coal']
In-sample R-squared: 6.55% for Smoke.lead against ['Txtls', 'Carry', 'Mines', 'Coal', 'Oil', 'Util', 'Telcm', 'Servs', 'Paper', 'Trans', 'Fin']
In-sample R-squared: 5.05% for Games.lead against ['Books', 'Clths', 'Coal', 'Fin']
In-sample R-squared: 6.30% for Books.lead against ['Games', 'Books', 'Coal', 'Oil', 'Util', 'Servs', 'BusEq', 'Rtail', 'Fin']
In-sample R-squared: 2.97% for Hshld.lead against ['Clths', 'Coal', 'Rtail']
In-sample R-squared: 7.82% for Clths.lead against ['Books', 'Clths', 'Chems', 'Steel', 'ElcEq', 'Carry', 'Coal', 'Oil', 'Util', 'Telcm', 'Servs', 'BusEq', 'Rtail']
In-sample R-squared: 2.68% for Hlth.lead against ['Books', 'Mines', 'Coal', 'Util']
In-sample R-squared: 0.78% for Chems.lead against ['Clths']
In-sample R-squared: 7.91% for Txtls.lead against ['Clths', 'Autos', 'Coal', 'Oil', 'Rtail', 'Fin']
In

0.03862278631691256

In [12]:
# use all predictors - higher in-sample R-squared
coef_dict_all = []
for _ in responses:
    coef_dict_all.append(range(len(predictors)))
predict_with_subsets(X, Y, LinearRegression, coef_dict_all, verbose=False)


0.0663748688823735

In [13]:
# first iteration will train up to including 196911
# will use 196912 to predict 197001
# 1970101 will be first month of performance to use
# train on first 121 months up to 196912 (0:120), put first prediction in P[121] (122nd row)
# first month of performance will be 197002
FIRST_TRAIN_MONTHS = 121
FIRST_PREDICT_MONTH = FIRST_TRAIN_MONTHS # This is stupid but keeps my head straight

print(X[FIRST_TRAIN_MONTHS])
print(data.iloc[FIRST_TRAIN_MONTHS][:30])

[ -3.34  -1.95  -7.59  -7.76 -12.05  -7.5   -5.69  -7.71  -7.37  -5.26
  -9.84  -6.31  -7.15  -6.89  -9.35 -12.49  -2.34  -0.77 -12.16  -4.83
  -3.16 -11.17  -9.73  -8.89  -8.17  -8.28  -6.31 -13.12  -9.78  -6.2 ]
Food     -3.34
Beer     -1.95
Smoke    -7.59
Games    -7.76
Books   -12.05
Hshld    -7.50
Clths    -5.69
Hlth     -7.71
Chems    -7.37
Txtls    -5.26
Cnstr    -9.84
Steel    -6.31
FabPr    -7.15
ElcEq    -6.89
Autos    -9.35
Carry   -12.49
Mines    -2.34
Coal     -0.77
Oil     -12.16
Util     -4.83
Telcm    -3.16
Servs   -11.17
BusEq    -9.73
Paper    -8.89
Trans    -8.17
Whlsl    -8.28
Rtail    -6.31
Meals   -13.12
Fin      -9.78
Other    -6.20
Name: 197001, dtype: float64


In [14]:
class PredictWrapper():
    """Wrap an sklearn model e.g. LinearRegression to fit, predict all vars as a vector, 
    match the way our Keras model will do it"""

    def __init__(self, create_model, coef_dict):
        self.create_model = create_model
        self.coef_dict = coef_dict
        self.models = []
        
    def fit(self, X_fit, Y_fit, verbose=False):
        
        self.nrows, self.ncols = Y_fit.shape
        
        self.models = []
        # fit model for each column
        for responsecol in range(self.ncols):
            if not self.coef_dict[responsecol]:
                # don't fit
                self.models.append(None)
                continue
                
            # column indexes to fit against each other
            predcols = self.coef_dict[responsecol]
            if verbose:
                print("fitting on " + str(X_fit[:, predcols].shape) + str(predcols))
            model = self.create_model()
            model.fit(X_fit[:, predcols], Y_fit[:,responsecol])
            self.models.append(model)
                
            #debug
            #print(responsecol)
            #print(X_fit[:, predcols])
            #print("=====")
            #print(Y_fit[:,responsecol])
            #print("=====")
            #print(self.model.coef_)
            #print(self.model.intercept_)
            #print("=====")                

    def predict(self, X_predict, verbose=False):

        self.nrows, self.ncols = X_predict.shape

        predictions = []

        for responsecol in range(self.ncols):
            if not self.coef_dict[responsecol]:
                # don't predict
                #print('skip')
                predictions.append([np.nan])
                continue
        
            if verbose:
                print("predicting on" + str(X_predict[:, predcols].shape) + str(predcols))

            predcols = self.coef_dict[responsecol]
            y_pred = self.models[responsecol].predict(X_predict[:,predcols])
            predictions.append(y_pred)
        
        return np.array(predictions).transpose()
        
        

In [15]:
class BacktestModel():
    
    def __init__(self, 
                 X, # predictors
                 Y, # responses
                 create_model, # sklearn function to initialize model e.g. LinearRegression
                 coef_dict_param="all", # mapping of predictors to responses ("all", "timestep", or a list of lists)
                 startindex=FIRST_TRAIN_MONTHS,
                 scaler=None):
        
        Xrows, Xcols = X.shape
        Yrows, Ycols = Y.shape
        
        if Xrows != Yrows or Xcols != Ycols:
            raise(ValueError, "Shapes differ: X %s, Y %s" % (str(X.shape), str(Y.shape)))            
            
        self.X = X
        self.Y = Y
        self.Xscale = X.copy()
        self.Yscale = Y.copy()

        if scaler:
            # MinMaxScaler: each row (min->0, max->1) 
            # StandardScaler: each row (mean->0, SD->1)            
            # transpose, scale, transpose back because scales by columns
            print("scaler: %s " %str(scaler))
            self.Xscale = scaler().fit_transform(Xscale.transpose()).transpose()
            self.Yscale = scaler().fit_transform(Yscale.transpose()).transpose()
        
        self.create_model=create_model
        self.nrows, self.ncols = X.shape
        self.coef_dict_param = coef_dict_param
        self.startindex = startindex

    def fit_predict(self, ntrain, npredict=1, verbose=False):
        """for backtest, train model using Y v. X 
        train on first ntrain rows. if ntrain=121, fit 0:120
        predict following npredict rows 
        if npredict=1, predict row 121
        if npredict=12, predict rows 121-132
        """
        # fit first ntrain rows
        X_fit = self.Xscale[:ntrain]  # e.g. 0:120
        Y_fit = self.Yscale[:ntrain]
        # predict npredict rows
        X_predict = self.Xscale[ntrain:ntrain+npredict] # 121-122
        X_predict = X_predict.reshape(npredict,self.ncols)
       
        # if no coef_dict select predictors into coef_dict
        if self.coef_dict_param == "timestep":
            msg = "Performing subset selection"
            coef_dict = subset_selection(X_fit, Y_fit, LassoLarsIC(criterion='aic'))
        # if coef_dict == "all" use all predictors for each response        
        elif self.coef_dict_param == 'all':
            msg = "Using all predictors"
            coef_dict = [range(self.ncols) for _ in range(self.ncols)]
        else: # should check valid dict
            msg = "Using coef_dict predictors"
            coef_dict = self.coef_dict_param
        if verbose: 
            print(msg)
#            print(coef_dict)

        modelwrapper = PredictWrapper(self.create_model, coef_dict)
        modelwrapper.fit(X_fit, Y_fit)
        return modelwrapper.predict(X_predict)

    # predict all months
    # initial train_months = 120 -> train first model on 120 rows
    # first prediction will be in P[120] (121st row)
    # step = 6 -> predict following 6 rows, then step forward 6 months at a time
    # initialize predictions matrix self.P
    
    # use either step or folds
    # step, do range(self.startindex, nrows, step)
    # folds, at each fold train 0:startfold, predict startfold+1:endfold
    # store only out-of-sample predictions in P, calc out-of-sample MSE
    
    # using a step > 1 or folds is quicker, for quicker xval, or to speed up by not estimating model at each timestep

    def gen_predictions(self,
                        step=1, 
                        splits=None,
                        verbose=False):

        self.P = np.zeros_like(self.Y)

        progress_i = 0
        self.nrows, self.ncols = Y.shape
        
        if splits:
            month_indexes = splits[:-1] # last index is nrows
        else:
            # create list of steps
            month_indexes = list(range(self.startindex, nrows, step))
        steps = [month_indexes[i+1]-month_indexes[i] for i in range(len(month_indexes)-1)]
        # last step -> end
        steps.append(self.nrows - month_indexes[-1])
        
        if verbose:
            print ("Steps: " + str(month_indexes))

        for month_index, forecast_rows in zip(month_indexes, steps):
            if verbose:
                print("Training on first %d rows (%d:%d), putting predictions in rows %s" % (month_index, 
                                                                                            0, month_index-1, 
                                                                                            str(range(month_index,month_index+forecast_rows))))
            predictions = self.fit_predict(month_index, forecast_rows, verbose=verbose)
            
            first_pred_row = month_index
            for row_index in range(forecast_rows):
                self.P[first_pred_row+row_index] = predictions[row_index]
            sys.stdout.write('.')
            progress_i += 1
            if progress_i % 80 == 0:
                print("")
                print("%s Still training step %d of %d" % (time.strftime("%H:%M:%S"), progress_i, len(month_indexes)))
            sys.stdout.flush()
        print("")
        
        msetemp = (self.P[self.startindex:]-self.Yscale[self.startindex:])**2
        #remove nans
        msetemp = msetemp[~np.isnan(msetemp)]
        mse = np.mean(msetemp)
        print("MSE across all predictions: %.4f" % mse)
        # force unpredicted ys to be nans, then remove nans
        vartemp = self.Yscale[self.startindex:] - self.P[self.startindex:] + self.P[self.startindex:]
        vartemp = vartemp[~np.isnan(vartemp)]
        y_variance = np.var(vartemp[self.startindex:])
        print("Variance: %.4f" % (y_variance))
        print("R-squared: %.4f" % (1- mse/y_variance))
        return mse

    def walkforward_xval (self, n_splits=5, verbose=False):
        """quick and dirty genreturns, with a step"""
        # generate k-folds
        kf = KFold(n_splits=n_splits)
        kf.get_n_splits(X)
        last_indexes = []
        for train_index, test_index in kf.split(X):
            # use test_index as last index to train
            last_index = test_index[-1] + 1
            last_indexes.append(last_index)
        print("%s Generate splits %s" % (time.strftime("%H:%M:%S"), str([i for i in last_indexes])))
        return self.gen_predictions(splits=last_indexes, verbose=verbose)
    
    def gen_returns(self, port_returns_func, verbose=False):

        self.R = np.zeros(self.P.shape[0])
        nrows, ncols = self.P.shape
        first_pred_month=self.startindex
        
        indcount = [0] * ncols
        longcount = [0] * ncols
        shortcount = [0] * ncols
        
        for month_index in range(first_pred_month, nrows-1):
            return_month = month_index + 1
            port_return, long_indexes, short_indexes = port_returns_func(self.P[month_index], 
                                                                         self.X[return_month])
            self.R[return_month] = port_return
            
            for i in long_indexes:
                indcount[i] += 1
                longcount[i] += 1
            for i in short_indexes:
                indcount[i] += 1
                shortcount[i] += 1
                
        for i in range(len(predictors)):
            print("%s: long %d times, short %d times, total %d times" % (predictors[i], 
                                                                         longcount[i], 
                                                                         shortcount[i], 
                                                                         indcount[i]))
        return self.R

    def report_returns(self, start_date='01/01/1970', freq='M'):

        first_pred_month=self.startindex        
        results = self.R[first_pred_month:]
        index = pd.date_range(start_date,periods=results.shape[0], freq=freq)
        perfdata = pd.DataFrame(results,index=index,columns=['Returns'])
        perfdata['Equity'] = 100 * np.cumprod(1 + results / 100)
        self.cumulative_return = perfdata['Equity']

        stats = perfdata['Equity'].calc_stats()
        
        retframe = pd.DataFrame([stats.stats.loc['start'],
                                 stats.stats.loc['end'],
                                 stats.stats.loc['cagr'],
                                 stats.stats.loc['yearly_vol'],
                                 stats.stats.loc['yearly_sharpe'],
                                 stats.stats.loc['max_drawdown'],
                                 ffn.core.calc_sortino_ratio(perfdata.Returns, rf=0, nperiods=564, annualize=False),
                                ],
                                index = ['start',
                                         'end',
                                         'cagr',
                                         'yearly_vol',
                                         'yearly_sharpe',
                                         'max_drawdown',
                                         'sortino',
                                        ],
                                columns=['Value'])   
        return retframe

In [16]:
NUMSTOCKS = 6 # top quintile (and bottom)

def calc_returns(prediction_row, return_row, numstocks=NUMSTOCKS, verbose=False):

    # ensure nan sorts to top for shorts
    short_sort_array = [999999 if np.isnan(x) else x for x in prediction_row]
    # pick bottom numstocks
    select_array = np.argsort(short_sort_array)
    short_indexes = select_array[:numstocks]

    # ensure nan sorts to bottom for longs
    long_sort_array = [-999999 if np.isnan(x) else x for x in prediction_row]
    # pick top numstocks
    select_array = np.argsort(long_sort_array)
    long_indexes = select_array[-numstocks:]
    
    if verbose:
        print("Longs: %s" %(str([(i,prediction_row[i]) for i in long_indexes])))
        print("Shorts: %s" %(str([(i,prediction_row[i]) for i in short_indexes])))

    # compute equal weighted long/short return
    return np.mean(return_row[long_indexes])/2 - np.mean(return_row[short_indexes])/2, long_indexes, short_indexes


In [17]:
start_date_int = data.index[FIRST_TRAIN_MONTHS]
start_year, start_month = start_date_int // 100, start_date_int % 100
start_date_str = "%02d/%02d/%d" % (start_month, 1, start_year)
start_date_str

'01/01/1970'

In [18]:

#coef_dict = subset_selection(X, Y, LassoLarsIC(criterion='aic'))

backtestmodel = BacktestModel(X, Y, LinearRegression, coef_dict_param=coef_dict_paper, startindex=FIRST_TRAIN_MONTHS)
backtestmodel.gen_predictions(verbose=False)
backtestmodel.gen_returns(calc_returns, verbose=False)
backtestmodel.report_returns(start_date=start_date_str, freq='M')


................................................................................
13:38:58 Still training step 80 of 563
................................................................................
13:38:59 Still training step 160 of 563
................................................................................
13:39:00 Still training step 240 of 563
................................................................................
13:39:01 Still training step 320 of 563
................................................................................
13:39:02 Still training step 400 of 563
................................................................................
13:39:03 Still training step 480 of 563
................................................................................
13:39:04 Still training step 560 of 563
...
MSE across all predictions: 38.7326
Variance: 38.6398
R-squared: -0.0024
Food: long 102 times, short 40 times, total 142 times
Beer: long 128 times, s

,Value
start,1970-01-31 00:00:00
end,2016-11-30 00:00:00
cagr,0.0654971
yearly_vol,0.0816703
yearly_sharpe,0.815644
max_drawdown,-0.0911841
sortino,0.622885


In [19]:
def mychart(args, names=None):
    x_coords = np.linspace(1970, 2016, args[0].shape[0])
    
    plotdata = []
    for i in range(len(args)):
        tracelabel = "Trace %d" % i
        if names:
                tracelabel=names[i]
        plotdata.append(Scatter(x=x_coords,
                                y=args[i].reshape(-1),
                                mode = 'line',
                                name=tracelabel))    

    layout = Layout(
        autosize=False,
        width=600,
        height=480,
        yaxis=dict(
            type='log',
            autorange=True
        )
    )
    
    fig = Figure(data=plotdata, layout=layout)
    
    return iplot(fig)
    


In [20]:
perf_post_LASSO = backtestmodel.cumulative_return
mychart([perf_post_LASSO],["Post-LASSO"])


In [21]:
# do subset selection at each timestep 
backtestmodel = BacktestModel(X, Y, LinearRegression, coef_dict_param="timestep", startindex=FIRST_TRAIN_MONTHS)
backtestmodel.gen_predictions(verbose=False)
backtestmodel.gen_returns(calc_returns, verbose=False)
backtestmodel.report_returns(start_date=start_date_str, freq='M')


................................................................................
02:03:09 Still training step 80 of 563
................................................................................
02:03:21 Still training step 160 of 563
................................................................................
02:03:33 Still training step 240 of 563
................................................................................
02:03:44 Still training step 320 of 563
................................................................................
02:03:56 Still training step 400 of 563
................................................................................
02:04:08 Still training step 480 of 563
................................................................................
02:04:20 Still training step 560 of 563
...
MSE across all predictions: 41.4734
Variance: 39.4097
R-squared: -0.0524
Food: long 131 times, short 68 times, total 199 times
Beer: long 112 times, s

,Value
start,1970-01-31 00:00:00
end,2016-11-30 00:00:00
cagr,0.0355146
yearly_vol,0.0476641
yearly_sharpe,0.760555
max_drawdown,-0.128334
sortino,0.329118


In [22]:
perf_LASSO_each_timestep = backtestmodel.cumulative_return
mychart([perf_LASSO_each_timestep],["LASSO each timestep"])


In [23]:
# use all predictors at each timestep 
backtestmodel = BacktestModel(X, Y, LinearRegression, coef_dict_param="all", startindex=FIRST_TRAIN_MONTHS)
backtestmodel.gen_predictions(verbose=False)
backtestmodel.gen_returns(calc_returns, verbose=False)
backtestmodel.report_returns(start_date=start_date_str, freq='M')


................................................................................
02:04:22 Still training step 80 of 563
................................................................................
02:04:24 Still training step 160 of 563
................................................................................
02:04:25 Still training step 240 of 563
................................................................................
02:04:27 Still training step 320 of 563
................................................................................
02:04:29 Still training step 400 of 563
................................................................................
02:04:31 Still training step 480 of 563
................................................................................
02:04:32 Still training step 560 of 563
...
MSE across all predictions: 43.8692
Variance: 39.4097
R-squared: -0.1132
Food: long 111 times, short 69 times, total 180 times
Beer: long 131 times, s

,Value
start,1970-01-31 00:00:00
end,2016-11-30 00:00:00
cagr,0.0278631
yearly_vol,0.0601275
yearly_sharpe,0.489589
max_drawdown,-0.16111
sortino,0.222994


In [24]:
perf_all_preds = backtestmodel.cumulative_return
mychart([perf_all_preds],["All preds"])


In [25]:
mychart([perf_post_LASSO, perf_LASSO_each_timestep, perf_all_preds],["Post-LASSO", "LASSO Each Timestep", "OLS All Predictors"])

In [26]:
# double check results_post_LASSO
#model = LinearRegression()
#R = run_backtest(X, Y, model, coef_dict_paper, startmonth=STARTMONTH, summary=False)
results_post_LASSO = R[FIRST_PREDICT_MONTH:]
print(len(results_post_LASSO))
#print(results_post_LASSO)
print(np.mean(results_post_LASSO))
print(np.std(results_post_LASSO) * np.sqrt(12))
print(np.prod(1 + results_post_LASSO / 100))
print(np.prod(1 + results_post_LASSO / 100) ** (12.0/len(results_post_LASSO))-1)

NameError: name 'R' is not defined

In [27]:
backtestmodel = BacktestModel(X, Y, LinearRegression, coef_dict_param="timestep", startindex=FIRST_TRAIN_MONTHS)
backtestmodel.walkforward_xval(n_splits=5, verbose=True)


02:06:30 Generate splits [137, 274, 411, 548, 684]
Steps: [137, 274, 411, 548]
Training on first 137 rows (0:136), putting predictions in rows range(137, 274)
Performing subset selection
.Training on first 274 rows (0:273), putting predictions in rows range(274, 411)
Performing subset selection
.Training on first 411 rows (0:410), putting predictions in rows range(411, 548)
Performing subset selection
.Training on first 548 rows (0:547), putting predictions in rows range(548, 684)
Performing subset selection
.
MSE across all predictions: 41.5825
Variance: 39.4097
R-squared: -0.0551


41.58252783029197

In [28]:
backtestmodel.gen_returns(calc_returns, verbose=False)
retframe = backtestmodel.report_returns(start_date=start_date_str, freq='M')
retframe

Food: long 95 times, short 81 times, total 176 times
Beer: long 135 times, short 77 times, total 212 times
Smoke: long 196 times, short 128 times, total 324 times
Games: long 164 times, short 130 times, total 294 times
Books: long 107 times, short 84 times, total 191 times
Hshld: long 136 times, short 108 times, total 244 times
Clths: long 119 times, short 91 times, total 210 times
Hlth: long 115 times, short 106 times, total 221 times
Chems: long 62 times, short 130 times, total 192 times
Txtls: long 131 times, short 105 times, total 236 times
Cnstr: long 78 times, short 84 times, total 162 times
Steel: long 32 times, short 144 times, total 176 times
FabPr: long 41 times, short 95 times, total 136 times
ElcEq: long 116 times, short 88 times, total 204 times
Autos: long 87 times, short 136 times, total 223 times
Carry: long 131 times, short 106 times, total 237 times
Mines: long 133 times, short 90 times, total 223 times
Coal: long 196 times, short 153 times, total 349 times
Oil: long 

,Value
start,1970-01-31 00:00:00
end,2016-11-30 00:00:00
cagr,0.0278818
yearly_vol,0.0509348
yearly_sharpe,0.57354
max_drawdown,-0.184168
sortino,0.283544


In [29]:
retframe.loc['yearly_sharpe']

Value    0.57354
Name: yearly_sharpe, dtype: object

In [20]:
# closure to wrap model creation function - return a function to create model
def create_model(n_hidden_layers, layer_size, reg_penalty):
    def myclosure():
        return MLPRegressor(hidden_layer_sizes=tuple([layer_size]*n_hidden_layers),
                            alpha=reg_penalty,
                            activation='tanh',
                            max_iter=10000, 
                            tol=1e-10,
                            solver='lbfgs')
    return myclosure

# use all predictors at each timestep 
backtestmodel = BacktestModel(X, Y, create_model(3,2,1.0), coef_dict_param="all", startindex=FIRST_TRAIN_MONTHS)
backtestmodel.walkforward_xval(n_splits=5)

# MSE is a bit worse than LinearRegression

13:39:54 Generate splits [137, 274, 411, 548, 684]
....
MSE across all predictions: 48.6660
Variance: 39.4097
R-squared: -0.2349


48.66600636249855

In [31]:
MODELPREFIX = "MLP"

n_hiddens = [1, 2, 3]
layer_sizes = [1, 2, 4, 8]
reg_penalties = [0.0, 0.001, 0.01, 0.1, 1]
hyperparameter_combos = list(product(n_hiddens, layer_sizes, reg_penalties))

print("%s Running %d experiments" % (time.strftime("%H:%M:%S"), len(hyperparameter_combos)))
experiments = {}
sharpes = {}

for counter, param_list in enumerate(hyperparameter_combos):
    n_hidden_layers, layer_size, reg_penalty = param_list
    print("%s Running experiment %d of %d" % (time.strftime("%H:%M:%S"), counter+1, len(hyperparameter_combos)))
    key = (n_hidden_layers, layer_size, reg_penalty)
    print("%s n_hidden_layers = %d, hidden_layer_size = %d, reg_penalty = %.6f" % 
          (time.strftime("%H:%M:%S"), n_hidden_layers, layer_size, reg_penalty))
    
    experiment_model = BacktestModel(X, Y, create_model(n_hidden_layers,layer_size,reg_penalty), coef_dict_param="all", startindex=FIRST_TRAIN_MONTHS)
    score = experiment_model.walkforward_xval(n_splits=5)
    experiments[key] = score

    experiment_model.gen_returns(calc_returns, verbose=False)
    retframe = experiment_model.report_returns(start_date=start_date_str, freq='M')
    sharpe = retframe.loc['yearly_sharpe']
    sharpes[key] = sharpe.values[0]
    print("%s MSE: %f" % (str(key), score))
    print("%s Sharpe: %f" % (str(key), sharpe))



02:10:25 Running 60 experiments
02:10:25 Running experiment 1 of 60
02:10:25 n_hidden_layers = 1, hidden_layer_size = 1, reg_penalty = 0.000000
02:10:25 Generate splits [137, 274, 411, 548, 684]
....
MSE across all predictions: 42.1091
Variance: 39.4097
R-squared: -0.0685
Food: long 72 times, short 88 times, total 160 times
Beer: long 84 times, short 84 times, total 168 times
Smoke: long 233 times, short 79 times, total 312 times
Games: long 273 times, short 199 times, total 472 times
Books: long 197 times, short 150 times, total 347 times
Hshld: long 64 times, short 104 times, total 168 times
Clths: long 271 times, short 197 times, total 468 times
Hlth: long 101 times, short 81 times, total 182 times
Chems: long 37 times, short 128 times, total 165 times
Txtls: long 141 times, short 56 times, total 197 times
Cnstr: long 10 times, short 30 times, total 40 times
Steel: long 13 times, short 230 times, total 243 times
FabPr: long 36 times, short 113 times, total 149 times
ElcEq: long 120 

....
MSE across all predictions: 46.1546
Variance: 39.4097
R-squared: -0.1711
Food: long 100 times, short 115 times, total 215 times
Beer: long 131 times, short 109 times, total 240 times
Smoke: long 160 times, short 115 times, total 275 times
Games: long 158 times, short 141 times, total 299 times
Books: long 99 times, short 77 times, total 176 times
Hshld: long 125 times, short 101 times, total 226 times
Clths: long 113 times, short 101 times, total 214 times
Hlth: long 85 times, short 84 times, total 169 times
Chems: long 103 times, short 125 times, total 228 times
Txtls: long 112 times, short 86 times, total 198 times
Cnstr: long 77 times, short 83 times, total 160 times
Steel: long 76 times, short 156 times, total 232 times
FabPr: long 98 times, short 133 times, total 231 times
ElcEq: long 110 times, short 121 times, total 231 times
Autos: long 108 times, short 156 times, total 264 times
Carry: long 103 times, short 91 times, total 194 times
Mines: long 168 times, short 137 times,

....
MSE across all predictions: 54.4354
Variance: 39.4097
R-squared: -0.3813
Food: long 94 times, short 108 times, total 202 times
Beer: long 133 times, short 89 times, total 222 times
Smoke: long 136 times, short 105 times, total 241 times
Games: long 149 times, short 105 times, total 254 times
Books: long 109 times, short 49 times, total 158 times
Hshld: long 89 times, short 89 times, total 178 times
Clths: long 115 times, short 97 times, total 212 times
Hlth: long 86 times, short 115 times, total 201 times
Chems: long 64 times, short 119 times, total 183 times
Txtls: long 114 times, short 98 times, total 212 times
Cnstr: long 109 times, short 107 times, total 216 times
Steel: long 92 times, short 163 times, total 255 times
FabPr: long 98 times, short 136 times, total 234 times
ElcEq: long 123 times, short 142 times, total 265 times
Autos: long 82 times, short 105 times, total 187 times
Carry: long 113 times, short 95 times, total 208 times
Mines: long 165 times, short 136 times, to

....
MSE across all predictions: 71.4139
Variance: 39.4097
R-squared: -0.8121
Food: long 92 times, short 80 times, total 172 times
Beer: long 148 times, short 87 times, total 235 times
Smoke: long 146 times, short 110 times, total 256 times
Games: long 165 times, short 118 times, total 283 times
Books: long 135 times, short 125 times, total 260 times
Hshld: long 95 times, short 106 times, total 201 times
Clths: long 144 times, short 133 times, total 277 times
Hlth: long 85 times, short 87 times, total 172 times
Chems: long 99 times, short 110 times, total 209 times
Txtls: long 118 times, short 82 times, total 200 times
Cnstr: long 85 times, short 86 times, total 171 times
Steel: long 107 times, short 146 times, total 253 times
FabPr: long 87 times, short 104 times, total 191 times
ElcEq: long 122 times, short 96 times, total 218 times
Autos: long 90 times, short 128 times, total 218 times
Carry: long 122 times, short 110 times, total 232 times
Mines: long 131 times, short 112 times, to

....
MSE across all predictions: 41.4351
Variance: 39.4097
R-squared: -0.0514
Food: long 156 times, short 91 times, total 247 times
Beer: long 128 times, short 81 times, total 209 times
Smoke: long 142 times, short 62 times, total 204 times
Games: long 269 times, short 162 times, total 431 times
Books: long 105 times, short 82 times, total 187 times
Hshld: long 95 times, short 121 times, total 216 times
Clths: long 188 times, short 142 times, total 330 times
Hlth: long 87 times, short 89 times, total 176 times
Chems: long 19 times, short 134 times, total 153 times
Txtls: long 164 times, short 107 times, total 271 times
Cnstr: long 74 times, short 140 times, total 214 times
Steel: long 53 times, short 194 times, total 247 times
FabPr: long 47 times, short 135 times, total 182 times
ElcEq: long 14 times, short 20 times, total 34 times
Autos: long 33 times, short 68 times, total 101 times
Carry: long 85 times, short 64 times, total 149 times
Mines: long 213 times, short 122 times, total 3

...

....
MSE across all predictions: 86.2131
Variance: 39.4097
R-squared: -1.1876
Food: long 95 times, short 106 times, total 201 times
Beer: long 127 times, short 110 times, total 237 times
Smoke: long 142 times, short 128 times, total 270 times
Games: long 143 times, short 106 times, total 249 times
Books: long 89 times, short 123 times, total 212 times
Hshld: long 112 times, short 107 times, total 219 times
Clths: long 103 times, short 82 times, total 185 times
Hlth: long 109 times, short 92 times, total 201 times
Chems: long 88 times, short 123 times, total 211 times
Txtls: long 140 times, short 86 times, total 226 times
Cnstr: long 112 times, short 102 times, total 214 times
Steel: long 89 times, short 139 times, total 228 times
FabPr: long 101 times, short 102 times, total 203 times
ElcEq: long 133 times, short 115 times, total 248 times
Autos: long 109 times, short 101 times, total 210 times
Carry: long 128 times, short 117 times, total 245 times
Mines: long 129 times, short 103 tim

....
MSE across all predictions: 41.0327
Variance: 39.4097
R-squared: -0.0412
Food: long 166 times, short 137 times, total 303 times
Beer: long 190 times, short 110 times, total 300 times
Smoke: long 195 times, short 38 times, total 233 times
Games: long 314 times, short 185 times, total 499 times
Books: long 103 times, short 119 times, total 222 times
Hshld: long 50 times, short 83 times, total 133 times
Clths: long 159 times, short 158 times, total 317 times
Hlth: long 93 times, short 69 times, total 162 times
Chems: long 42 times, short 156 times, total 198 times
Txtls: long 172 times, short 84 times, total 256 times
Cnstr: long 45 times, short 109 times, total 154 times
Steel: long 4 times, short 249 times, total 253 times
FabPr: long 30 times, short 147 times, total 177 times
ElcEq: long 18 times, short 46 times, total 64 times
Autos: long 80 times, short 128 times, total 208 times
Carry: long 97 times, short 83 times, total 180 times
Mines: long 226 times, short 93 times, total 3

....
MSE across all predictions: 45.8784
Variance: 39.4097
R-squared: -0.1641
Food: long 103 times, short 114 times, total 217 times
Beer: long 165 times, short 139 times, total 304 times
Smoke: long 181 times, short 120 times, total 301 times
Games: long 125 times, short 126 times, total 251 times
Books: long 133 times, short 85 times, total 218 times
Hshld: long 82 times, short 124 times, total 206 times
Clths: long 133 times, short 142 times, total 275 times
Hlth: long 90 times, short 142 times, total 232 times
Chems: long 82 times, short 110 times, total 192 times
Txtls: long 149 times, short 75 times, total 224 times
Cnstr: long 83 times, short 77 times, total 160 times
Steel: long 71 times, short 160 times, total 231 times
FabPr: long 89 times, short 115 times, total 204 times
ElcEq: long 110 times, short 109 times, total 219 times
Autos: long 62 times, short 77 times, total 139 times
Carry: long 129 times, short 89 times, total 218 times
Mines: long 108 times, short 70 times, to

....
MSE across all predictions: 56.9358
Variance: 39.4097
R-squared: -0.4447
Food: long 89 times, short 97 times, total 186 times
Beer: long 76 times, short 82 times, total 158 times
Smoke: long 144 times, short 140 times, total 284 times
Games: long 159 times, short 73 times, total 232 times
Books: long 118 times, short 67 times, total 185 times
Hshld: long 90 times, short 121 times, total 211 times
Clths: long 121 times, short 141 times, total 262 times
Hlth: long 85 times, short 106 times, total 191 times
Chems: long 65 times, short 118 times, total 183 times
Txtls: long 112 times, short 92 times, total 204 times
Cnstr: long 103 times, short 110 times, total 213 times
Steel: long 70 times, short 132 times, total 202 times
FabPr: long 117 times, short 100 times, total 217 times
ElcEq: long 117 times, short 122 times, total 239 times
Autos: long 109 times, short 129 times, total 238 times
Carry: long 141 times, short 134 times, total 275 times
Mines: long 174 times, short 119 times, 

....
MSE across all predictions: 83.8104
Variance: 39.4097
R-squared: -1.1266
Food: long 71 times, short 135 times, total 206 times
Beer: long 91 times, short 81 times, total 172 times
Smoke: long 136 times, short 122 times, total 258 times
Games: long 181 times, short 124 times, total 305 times
Books: long 106 times, short 133 times, total 239 times
Hshld: long 92 times, short 111 times, total 203 times
Clths: long 111 times, short 122 times, total 233 times
Hlth: long 108 times, short 109 times, total 217 times
Chems: long 103 times, short 101 times, total 204 times
Txtls: long 133 times, short 101 times, total 234 times
Cnstr: long 97 times, short 113 times, total 210 times
Steel: long 108 times, short 113 times, total 221 times
FabPr: long 99 times, short 110 times, total 209 times
ElcEq: long 83 times, short 105 times, total 188 times
Autos: long 129 times, short 88 times, total 217 times
Carry: long 123 times, short 113 times, total 236 times
Mines: long 135 times, short 112 time

In [32]:
# list and chart experiments
flatlist = [list(l[0]) + [l[1]] for l in experiments.items()]
 
lossframe = pd.DataFrame(flatlist, columns=["n_hidden_layers", "layer_size", "reg_penalty", "MSE"])
# one row didn't converge properly - messes up plotly scales
#for i in list(lossframe.loc[lossframe['loss']> 1000].index):
#    lossframe.at[i, 'loss'] = 100
lossframe.sort_values(['MSE'])

,n_hidden_layers,layer_size,reg_penalty,MSE
40,3,1,0.000,41.032666
41,3,1,0.001,41.315630
21,2,1,0.001,41.342142
42,3,1,0.010,41.353013
20,2,1,0.000,41.435145
1,1,1,0.001,41.735779
22,2,1,0.010,41.815551
0,1,1,0.000,42.109091
43,3,1,0.100,42.113443
23,2,1,0.100,42.202308


In [34]:
# list sharpes
flatlist = [list(l[0]) + [l[1]] for l in sharpes.items()]
 
sharpeframe = pd.DataFrame(flatlist, columns=["n_hidden_layers", "layer_size", "reg_penalty", "sharpe"])
# one row didn't converge properly - messes up plotly scales
#for i in list(lossframe.loc[lossframe['loss']> 1000].index):
#    lossframe.at[i, 'loss'] = 100
#list(l) +
sharpeframe.sort_values(['sharpe'])

,n_hidden_layers,layer_size,reg_penalty,sharpe
54,3,4,1.000,-0.079051
57,3,8,0.010,-0.074976
59,3,8,1.000,0.016428
34,2,4,1.000,0.030283
55,3,8,0.000,0.030570
56,3,8,0.001,0.035304
19,1,8,1.000,0.042440
49,3,2,1.000,0.045281
13,1,4,0.100,0.066885
51,3,4,0.001,0.077789


In [35]:
sharpes_array = sharpeframe.values[:,3]
print(sharpes_array.shape)
mses_array = lossframe.values[:,3]
print(mses_array.shape)

(60,)
(60,)


In [36]:
def myscatter(arg1, arg2, names=None):
    
    plotdata = []
    
    plotdata.append(Scatter(
        x = arg1,
        y = arg2,
        mode = 'markers'
    ))
    
    layout = Layout(
        autosize=False,
        width=600,
        height=480,
        yaxis=dict(
            type='log',
            autorange=True
        )
    )
    
    fig = Figure(data=plotdata, layout=layout)
    
    return iplot(fig)
    
myscatter(sharpes_array, mses_array)
# MSEs worse than linear regression
# no very good Sharpes
# weak correlation between MSEs, Sharpes, which is perplexing

In [37]:
# we can pick lowest loss , but first we look at patterns by hyperparameter
pd.DataFrame(lossframe.groupby(['n_hidden_layers'])['MSE'].mean())


,MSE
n_hidden_layers,
1,56.102615
2,58.143200
3,58.751386


In [38]:
pd.DataFrame(lossframe.groupby(['layer_size'])['MSE'].mean())


,MSE
layer_size,
1,41.945917
2,46.746841
4,59.069370
8,82.900807


In [39]:
pd.DataFrame(lossframe.groupby(['reg_penalty'])['MSE'].mean())


,MSE
reg_penalty,
0.000,55.944110
0.001,55.927929
0.010,57.537845
0.100,59.522436
1.000,59.396348


In [41]:
def plot_matrix(lossframe, x_labels, y_labels, x_suffix="", y_suffix=""):

    pivot = lossframe.pivot_table(index=[y_labels], columns=[x_labels], values=['MSE'])
#    print(pivot)
    # specify labels as strings, to force plotly to use a discrete axis
#    print(pivot.columns.levels[1]).values
#    print(lossframe[x_labels].dtype)
    
    if lossframe[x_labels].dtype == np.float64 or lossframe[x_labels].dtype == np.float32:
        xaxis = ["%f %s" % (i, x_suffix) for i in pivot.columns.levels[1].values]
    else:
        xaxis = ["%d %s" % (i, x_suffix) for i in pivot.columns.levels[1].values]
    if lossframe[y_labels].dtype == np.float64 or lossframe[y_labels].dtype == np.float32:
        yaxis = ["%f %s" % (i, y_suffix) for i in pivot.index.values]
    else:
        yaxis = ["%d %s" % (i, y_suffix) for i in pivot.index.values]
        
#    print(xaxis, yaxis)
    """plot a heat map of a matrix"""
    chart_width=640
    chart_height=480
    
    layout = Layout(
        title="%s v. %s" % (x_labels, y_labels),
        height=chart_height,
        width=chart_width,     
        margin=dict(
            l=150,
            r=30,
            b=120,
            t=100,
        ),
        xaxis=dict(
            title=x_labels,
            tickfont=dict(
                family='Arial, sans-serif',
                size=10,
                color='black'
            ),
        ),
        yaxis=dict(
            title=y_labels,
            tickfont=dict(
                family='Arial, sans-serif',
                size=10,
                color='black'
            ),
        ),
    )
    
    data = [Heatmap(z=pivot.values,
                    x=xaxis,
                    y=yaxis,
                    colorscale=[[0, 'rgb(0,0,255)', [1, 'rgb(255,0,0)']]],
                   )
           ]

    fig = Figure(data=data, layout=layout)
    return iplot(fig, link_text="")

plot_matrix(lossframe, "n_hidden_layers", "layer_size", x_suffix=" layers", y_suffix=" units")




In [42]:
plot_matrix(lossframe, "n_hidden_layers", "reg_penalty", x_suffix=" layers", y_suffix=" p")


In [43]:
plot_matrix(lossframe, "reg_penalty", "layer_size", x_suffix=" p", y_suffix="units")


In [21]:
# try best MSE 
backtestmodel = BacktestModel(X, Y, create_model(3,1,0.0), coef_dict_param="all", startindex=FIRST_TRAIN_MONTHS)
backtestmodel.gen_predictions(verbose=False)
backtestmodel.gen_returns(calc_returns, verbose=False)
backtestmodel.report_returns(start_date=start_date_str, freq='M')


................................................................................
13:47:28 Still training step 80 of 563
................................................................................
13:51:00 Still training step 160 of 563
................................................................................
13:54:40 Still training step 240 of 563
................................................................................
13:58:18 Still training step 320 of 563
................................................................................
14:01:50 Still training step 400 of 563
................................................................................
14:05:21 Still training step 480 of 563
................................................................................
14:08:35 Still training step 560 of 563
...
MSE across all predictions: 41.0652
Variance: 39.4097
R-squared: -0.0420
Food: long 68 times, short 52 times, total 120 times
Beer: long 103 times, sh

,Value
start,1970-01-31 00:00:00
end,2016-11-30 00:00:00
cagr,0.0203304
yearly_vol,0.0479585
yearly_sharpe,0.424542
max_drawdown,-0.134092
sortino,0.186696
